NOTE: this notebook runs the DESeq2 package using an R kernel - to run this notebook you will have to have the IRkernel (https://github.com/IRkernel/IRkernel) installed and runnable through Jupyter

# Load Data

## Counts

In [1]:
counts <- as.matrix(read.csv('../../data/precise1k/counts.csv',row.names=1))
head(counts)

,p1k_00860,p1k_00918,p1k_00081,p1k_00870,p1k_00091,p1k_00908,p1k_00715,p1k_00705,p1k_00140,p1k_00038,⋯,p1k_00404,p1k_00414,p1k_00251,p1k_00329,p1k_00241,p1k_00339,p1k_00390,p1k_00380,p1k_00817,p1k_00807
b0001,490,121,2259,67,43,205,2655,6685,84,7,⋯,218,396,256,366,428,340,149,47,136,281
b0002,11984,20898,16815,4290,36971,17356,17787,37218,36743,6705,⋯,16325,31478,1917,47270,7889,55221,22502,2026,38141,37186
b0003,4856,5335,4436,1542,14298,4184,5298,10128,11192,2219,⋯,4029,9467,670,15018,2379,16468,6458,798,8724,8793
b0004,6271,5154,7317,2027,12356,3751,8225,11990,13889,2517,⋯,5147,15108,875,18588,3248,19295,9486,911,11351,10414
b0005,372,390,91,95,311,679,393,571,296,24,⋯,162,462,19,285,96,237,247,196,225,240
b0006,456,430,749,406,657,490,453,933,847,311,⋯,791,885,174,1399,234,6281,579,298,680,774


## Metadata

In [2]:
metadata <- read.csv('../../data/precise1k/metadata_qc.csv',row.names=1)
head(metadata)

,sample_id,study,project,condition,rep_id,Strain.Description,Strain,Culture.Type,Evolved.Sample,Base.Media,⋯,R2,contact,creator,passed_fastqc,passed_pct_reads_mapped,passed_reads_mapped_to_CDS,passed_global_correlation,full_name,passed_similar_replicates,passed_number_replicates
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
p1k_00001,control__wt_glc__1,Control,control,wt_glc,1,Escherichia coli K-12 MG1655,MG1655,Batch,No,M9,⋯,s3://precise-db/raw_data/control/MG1655-M9-glucose-1_S1_L001_R2_001.fastq.gz,Sangwoo Seo,Sangwoo Seo,True,True,True,True,control:wt_glc,True,True
p1k_00002,control__wt_glc__2,Control,control,wt_glc,2,Escherichia coli K-12 MG1655,MG1655,Batch,No,M9,⋯,s3://precise-db/raw_data/control/MG1655-M9-glucose-2_S2_L001_R2_001.fastq.gz,Sangwoo Seo,Sangwoo Seo,True,True,True,True,control:wt_glc,True,True
p1k_00003,fur__wt_dpd__1,Fur,fur,wt_dpd,1,Escherichia coli K-12 MG1655,MG1655,Batch,No,M9,⋯,s3://precise-db/raw_data/fur/WTDPD1_S1_L001_R2_001.fastq.gz,Sangwoo Seo,Sangwoo Seo,True,True,True,True,fur:wt_dpd,True,True
p1k_00004,fur__wt_dpd__2,Fur,fur,wt_dpd,2,Escherichia coli K-12 MG1655,MG1655,Batch,No,M9,⋯,s3://precise-db/raw_data/fur/WTDPD2_S1_L001_R2_001.fastq.gz,Sangwoo Seo,Sangwoo Seo,True,True,True,True,fur:wt_dpd,True,True
p1k_00005,fur__wt_fe__1,Fur,fur,wt_fe,1,Escherichia coli K-12 MG1655,MG1655,Batch,No,M9,⋯,s3://precise-db/raw_data/fur/WT-Fe2-1_S1_L001_R2_001.fastq.gz,Sangwoo Seo,Sangwoo Seo,True,True,True,True,fur:wt_fe,True,True
p1k_00006,fur__wt_fe__2,Fur,fur,wt_fe,2,Escherichia coli K-12 MG1655,MG1655,Batch,No,M9,⋯,s3://precise-db/raw_data/fur/WT-FE2-2_S2_L001_R2_001.fastq.gz,Sangwoo Seo,Sangwoo Seo,True,True,True,True,fur:wt_fe,True,True


## Gene Table

In [3]:
geneTable <- read.csv('../../data/annotation/gene_info.csv', row.names=1)
head(geneTable)

,gene_name,synonyms,gene_product,COG,regulator,uniprot,start,end,strand,essential,⋯,y_ome,k_eff,schmidt_prot_med,heckmann_prot_med,proteomics,p1k,p1k_ctrl_log_tpm,p1k_median_log_tpm,p1k_mad_log_tpm,in_iM
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
b0001,thrL,thrL,thr operon leader peptide,No COG annotation,,P0AD86,190,255,+,False,⋯,False,False,NA,NA,False,False,10.372833,8.862343,1.9369518,False
b0002,thrA,thrA;thrA1;thrA2,fused aspartate kinase/homoserine dehydrogenase 1,Amino acid transport and metabolism,"RpoD,Ile-tRNA,Thr-tRNA,DksA,Fnr,ArcA,ppGpp",P00561,337,2799,+,False,⋯,False,True,4101.5,4434,True,True,10.314562,9.605329,1.0421427,True
b0003,thrB,thrB,homoserine kinase,Nucleotide transport and metabolism,"RpoD,Ile-tRNA,Thr-tRNA,DksA,ppGpp",P00547,2801,3733,+,False,⋯,False,True,442.0,1177,True,True,9.982251,9.232803,1.0356538,True
b0004,thrC,thrC,threonine synthase,Amino acid transport and metabolism,"RpoD,ppGpp,Ile-tRNA,Thr-tRNA,DksA",P00934,3734,5020,+,False,⋯,False,True,8818.5,3516,True,True,9.973666,9.274271,0.9062871,True
b0005,yaaX,yaaX,DUF2502 domain-containing protein YaaX,Function unknown,Lrp,P75616,5234,5530,+,False,⋯,True,False,NA,NA,False,True,7.132482,6.283661,1.0931462,True
b0006,yaaA,yaaA,peroxide stress resistance protein YaaA,Function unknown,,P0A8I3,5683,6459,-,False,⋯,True,True,539.5,333,True,True,6.494941,6.346626,0.4545190,True


## Select Counts

Only want counts for QC-passed data (things in metadata)

Only want counts for genes in P1K (to give fair comparison with DIMA results)

In [4]:
counts <- counts[rownames(geneTable[geneTable$p1k == 'True',]), rownames(metadata)]

In [5]:
all(colnames(counts) == rownames(metadata))

[1] TRUE

# Run DESeq

In [6]:
library(hash)
suppressPackageStartupMessages(library('DESeq2'))
library(stringr)

hash-2.2.6.2 provided by Decision Patterns




In [7]:
for (project in unique(metadata$project)) {
    print(project)
    # select only the metadata and counts for this project
    mdProject <- metadata[metadata$project == project, ]
    countsProject <- counts[,rownames(mdProject)]
    numCond <- length(unique(mdProject$condition))
    
    # only run DESeq if we have more than one condition and replicates
    if ((1 < numCond) & (numCond < length(mdProject$condition))) {
        
        # need to make sure the project reference is in this set - if not, need to go add it to the project metadata and counts
        projectRef <- unique(mdProject$project_reference)
        projectRefIDs <- strsplit(projectRef, ";")[[1]]
        refIDsPresent <- TRUE
        for (refID in projectRefIDs) {
            if (!(refID %in% rownames(mdProject))) {
                refIDsPresent = FALSE
                break
            }
        }
        # if we don't have the reference conditions, add them to the metadata and counts
        if (!refIDsPresent) {
            mdProject <- rbind(mdProject, metadata[projectRefIDs,])
            countsProject <- cbind(countsProject, counts[,projectRefIDs])
        }
        
        # add identifiers to the metadata to use as the design to avoid character issues with condition names
        unique_conds <- unique(mdProject$condition)
        cond_to_int_id <- hash()
        int_id_to_cond <- hash()
        for (i in 1:length(unique_conds)) {
            cond_to_int_id[[unique_conds[[i]]]] <- paste(i)
            int_id_to_cond[[paste(i)]] <- unique_conds[[i]]
        }
        cond_int_col <- c()
        for (cond in mdProject$condition) {
            cond_int_col <- append(cond_int_col, cond_to_int_id[[cond]])
        }
        mdProject <- cbind(mdProject, cond_int=cond_int_col)

        # create the DESeq dataset and then run DESeq
        deseqDataset <- DESeqDataSetFromMatrix(countData = countsProject,
                                  colData = mdProject,
                                  design = ~ cond_int)
        dds <- DESeq(deseqDataset)

        # get the results for each of the pairwise comparisons; save the resulting tables
        # can set cookCutoff FALSE because mostly have 2 replicates (applies with min 3)
        for (i in 1:length(unique_conds)) {
            for (j in 1:length(unique_conds)) {
                if (j > i) {
                    cond1 <- unique_conds[[i]]
                    cond2 <- unique_conds[[j]]
                    cond_int_1 <- cond_to_int_id[[cond1]]
                    cond_int_2 <- cond_to_int_id[[cond2]]
                    resultTable <- results(dds, contrast=c('cond_int', cond_int_1, cond_int_2), cooksCutoff=FALSE, alpha=0.01)
                    cond1_full <- paste(project, '__', cond1, sep='')
                    cond2_full <- paste(project, '__', cond2, sep='')
                    file_name <- paste(cond1_full, '_vs_', cond2_full, '.csv', sep='')
                    write.csv(resultTable, paste('../../data/precise1k/deg/',file_name, sep=''), row.names=TRUE)
                }
            }
        }
    }
}

[1] "control"
[1] "fur"


Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "acid"


Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "oxidative"


...

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "tcs"


Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "ros_tale"


Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

